# TAAS OPEN API : 고속도로구간별 도로위험지수정보 API
**ID : regenesis90@gmail.com / pw : ㄹㅣ제90별**

* http://taas.koroad.or.kr/api/selectHighwayDataSet.do
* 인증키 : vMrK%252BgbnL4byIqXRSFKtJy%252Fr6w6sQPPNt6Kz1RG6KvHnNQ%252BtWNaZ6xx1Fk8sEef

* `accident_merged.csv`에 있는 conzoneID별 도로위험지수만 뽑아내준다.

# Imports

In [37]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import json
import xml.etree.ElementTree as ET
import xmltodict
from tqdm import tqdm

import pandas as pd

# 기본정보 불러오기

In [38]:
section_info = pd.read_csv('data/Freeway_Section_list_TAAS.csv', encoding = 'cp949')
section_info.head(3)

,Unnamed: 0,Freeway_TAAS,Section_TAAS,도로단축명,콘존명
0,0,경부고속도로,건천IC-건천IC,경부선,건천IC-건천IC
1,1,경부고속도로,건천IC-경주IC,경부선,건천IC-경주IC
2,2,경부고속도로,건천IC-영천JC,경부선,건천IC-영천JC


In [39]:
accident_info = pd.read_csv('data/accident_merged.csv', encoding = 'cp949')
accident_info = pd.merge(accident_info, section_info, how = 'left', left_on = 'conzoneName', right_on = '콘존명').drop('도로단축명_y', axis = 'columns').drop('콘존명', axis = 'columns')

accident_info.head(3)

,Unnamed: 0_x,사고일자,사고일자.1,월별구분,사고시간,시간단위_3시간,주야구분,노선명,이정,방향,...,도로단축명_x,도로표출명,기점종점방향구분코드,시점명,종점명,conzoneID,conzoneName,Unnamed: 0_y,Freeway_TAAS,Section_TAAS
0,1,2016-01-01,20160101,01월,18:40:00,18 ~21,야간,광주대구선,141.7,대구,...,광주대구선,광주대구,E,광주,대구,0120CZE230,고령JC-옥포JC,304.0,광주대구고속도로,고령JC-옥포JC
1,1,2016-01-01,20160101,01월,18:40:00,18 ~21,야간,광주대구선,141.7,대구,...,광주대구선,광주대구,E,광주,대구,0120CZE230,고령JC-옥포JC,305.0,광주대구고속도로,고령JC-옥포JC
2,1,2016-01-01,20160101,01월,18:40:00,18 ~21,야간,광주대구선,141.7,대구,...,광주대구선,광주대구,E,광주,대구,0120CZE230,고령JC-옥포JC,306.0,광주대구고속도로,고령JC-옥포JC


In [40]:
frwy_sec = accident_info.loc[:, ['Freeway_TAAS', 'Section_TAAS']].drop_duplicates() # 중복행을 제거함
vehicle_type = ['01', '02', '03', '04']

# XML Parsing

In [41]:
# Service URL
xmlUrl = 'http://taas.koroad.or.kr/data/rest/road/dgdgr/highway'

My_API_Key = 'I666JO%2FXYCNYi%2Bx1hXtN1fSUdEOcDZX%2BHxA25Ye4JrkHpuATv0xgahcQk1%2FVml8p' # 사용자 인증키

#pages_1 = [i for i in range(1, 640)]
#pages_2 = [i for i in range(240, 359)]

# 필수 요청인자를 파라미터로 정리하기

* 반드시 OPEN API의 페이지를 확인해야 합니다

In [42]:
total_df = pd.DataFrame()

frwy_list = list(frwy_sec['Freeway_TAAS'])
sec_list = list(frwy_sec['Section_TAAS'])

for veh in vehicle_type: # 각 차종에 대하여

    for frwy, sec in tqdm(zip(frwy_list, sec_list)):
    
        fullUrl = xmlUrl + '?' + 'authKey=' + My_API_Key
    
        queryParams = '&' + urlencode(    # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것
            {
                quote_plus('frwyNm') : frwy,
                quote_plus('frwySctnNm') : sec,
                quote_plus('vhctyCd') : veh # 차종구분(01:승용차, 02:버스, 03:택시, 04:화물차)
            }
        )
    
        response = urllib.request.Request(fullUrl + queryParams)
        response_body = urlopen(response).read() #get bytes data

        decode_data = response_body.decode('utf-8')

        xml_parse = xmltodict.parse(decode_data)     # string인 xml 파싱
        xml_dict = json.loads(json.dumps(xml_parse))
        
        if xml_dict['response']['body']['items']['item']:
            df = pd.json_normalize(xml_dict['response']['body']['items']['item'])

        #if len(df) >= 1:
            df['Freeway'] = frwy
            df['Section'] = sec
            df['vehicle_type'] = veh
            total_df = pd.concat([total_df, df])
        
        else:
            pass
            #print(frwy, sec)

total_df.to_csv(f'data/Freeway_Danger_Index.csv', encoding = 'cp949')

179it [02:19,  1.28it/s]
179it [02:19,  1.29it/s]
179it [02:14,  1.33it/s]
179it [02:25,  1.23it/s]


In [44]:
total_df.to_csv(f'data/Freeway_Danger_Index.csv', encoding = 'cp949')